## Server 

In [ ]:
{
  "pipeline": "spacy_sklearn",
  "path" : "./",
  "data" : "./data/gstfaq-data.json"
}

In [ ]:
-m rasa_nlu.train -c config.json

In [ ]:
-m rasa_nlu.server -c config.json --server_model_dirs=./model_YYYYMMDD-HHMMSS

## Client 

In [ ]:
<!DOCTYPE HTML>
<html lang="en">
<head>
…
</head>
    <body>
         <h1 align="center">GST FAQ Chat</h1>
        <div class="container">
            {% block content %}{% endblock %}
        </div>
</body>
    <footer>
        {% block other_footers %}{% endblock %}
    </footer>
</html>

In [ ]:
{% block content %}
<div class="row">
    <div class="col-sm-6">
        <div class="row">
            <div class="chat_window">
                <ul class="messages"></ul>
                <div class="bottom_wrapper clearfix">
                    <div class="message_input_wrapper">
                        <input id="msg_input" class="message_input" placeholder="Say Hi to begin chat..." />
                    </div>
                    <div class="send_message">
                         <div class="text">send</div>
                    </div>
                </div>
            </div>
            <div class="message_template">
                <li class="message">
                    <div class="avatar"></div>
                    <div class="text_wrapper">
                        <div class="text"></div>
                    </div>
                </li>
            </div>
        </div>
    </div>
</div>
{% endblock %}


{% block other_footers %}
<script src="{{ url_for('static', filename="js/bind.js") }}"></script>
{% endblock %}

## Engine 

In [ ]:
intent_response_dict = {
    "intro": ["This is a GST FAQ bot. One stop-shop to all your GST related queries"],
    "greet":["hey","hello","hi"],
    "goodbye":["bye","It was nice talking to you","see you","ttyl"],
    "affirm":["cool","I know you would like it"],
    "faq_link":['You can check all the events here <a href="https://www.cbec.gov.in/resources//htdocs-cbec/deptt_offcr/faq-on-gst.pdf</a>']
}

In [ ]:
@app.route('/chat',methods=["POST"])
def chat():
    try:
        user_message = request.form["text"]
        response = requests.get("http://localhost:5000/parse",params={"q":user_message})
        response = response.json()
        response = response["topScoringIntent"]
        intent = response.get("intent")
        entities = response.get("entities")
        if intent == "gst-info":
            response_text = gst_info(entities
        elif intent == "gst-query":
            response_text = gst_query(entities)
        else:
            response_text = get_random_response(intent)
        return jsonify({"status":"success","response":response_text})
    except Exception as e:
        print(e)
        return jsonify({"status":"success","response":"Sorry I am not trained to do that yet..."})
